Maskiere die Emailadressen in der Spalte _LogonName_ der Tabelle _People_ mit der geeigneten Maskierungsfunktion.

In [ ]:
-- Email
ALTER TABLE Application.People
ALTER COLUMN LogonName ADD MASKED WITH (FUNCTION = 'email()')

Prüfe nun, ob die Daten mit dem User _ethan_ wirklich maskiert daherkommen.

In [ ]:
SELECT * FROM Application.People

Verschlüssle die Werte in der Spalte _CustomerID_ der Tabelle _CustomerTransactions_ und speichere diese in einer separaten Spalte. Verwende dabei den bereits erstellten Database Master Key.

In [ ]:
USE WideWorldImporters;

/*
falls noch nicht vorhanden, einen Database Master Key erstellen.
Der Key muss in der DB WideWorldImporters erstellt sein
*/
CREATE MASTER KEY ENCRYPTION BY PASSWORD = '123_MyComplexPassword';  

-- ein Zertifikat erstellen
CREATE CERTIFICATE CustomerIdentification
   WITH SUBJECT = 'Suppliers Bank Account Number';  
GO  

-- symmetrischen Key erstellen
CREATE SYMMETRIC KEY SSN_Key_01  
    WITH ALGORITHM = AES_256  
    ENCRYPTION BY CERTIFICATE CustomerIdentification;  
GO  

-- neue Spalte für die verschlüsselten Daten hinzufügen  
ALTER TABLE Newtable
    ADD EncryptedBAN varbinary(128);   
GO  

-- den symmetrischen Key öffnen, mit welchem die Daten verschlüsselt werden sollen  
OPEN SYMMETRIC KEY SSN_Key_01  
   DECRYPTION BY CERTIFICATE CustomerIdentification;  

/* 
die Werte in der Spalte BankAccountNumber mit dem symmetrischen Key SSN_Key_01
verschlüsseln und in der neuen Spalte EncryptedBAN speichern
*/ 
UPDATE Purchasing.CustomerTransactions  
SET CustomerIdentification = EncryptByKey(Key_GUID('SSN_Key_01'), CustomerID);  
GO  

Prüfe anschliessend, ob die neue, verschlüsselte Spalte wieder korrekt entschlüsselt werden kann.

In [ ]:
/*
Prüfen, ob die Verschlüsselung erfolgreich war 
bei der Ausgabe dieser Abfrage müssten nun die Werte der ersten Spalte mit den
Werten der dritten Spalte übereinstimmen 
*/
SELECT CustomerID, CustomerIdentification
    AS 'Encrypted CustomerID',  
    CONVERT(nvarchar, DecryptByKey(CustomerIdentification))   
    AS 'Decrypted CustomerID'  
    FROM Purchasing.CustomerTransactions;  
GO

-- Symmetrischen Key wieder schliessen
CLOSE SYMMETRIC KEY SSN_Key_01;
GO